# A Python tutorial on validating gVirtualXRay

**Purpose:** In this notebook, we aim to demonstrate that gVirtualXRay is able to generate analytic simulations on GPU comparable to images generated with the state-of-the-art Monte Caro simulation packages.

**Material and Methods:** To generate an X-ray image with an analytic simulation, we must solve the [Beer-Lambert law](https://en.wikipedia.org/wiki/Beer%E2%80%93Lambert_law), which relies on attenuation coefficients. The first step in validating gVirtualXRay is to ascertain that the attenuation coefficients used in gVirtualXRay are in perfect agreement with those found in the literature. Ground truth data is provided in the [NIST Standard Reference Database 126](https://www.nist.gov/pml/x-ray-mass-attenuation-coefficients).

The second step is to simulate an image with gVirtualXRay and compare it with a ground truth image. For this purpose, we use [Gate],(http://www.opengatecollaboration.org/) a wrapper for CERN's state-of-the-art Monte Caro simulation tool: [Geant4](https://geant4.web.cern.ch/).

In our simulation the source-to-object distance (SOD) is 1000mm, and the source-to-detector distance (SDD) is 1125mm. The beam spectrum is generated using [SpekPy](https://bitbucket.org/spekpy/spekpy_release/wiki/Home). We use a tube voltage of 90kV, which is typical of chest x-ray procedures for a 70kg male patient. The total aluminium equivalent filtration is 3.2mm.

### Simulation parameters

![](setup.png)

The sample is made of a 70x70x15mm box of water, in which 5 columns of 3 spheres of different radii (2, 3.5, and 5mm) have been inserted. A given material is associated to the spheres of each column (bone (cortical), blood (whole), brain (grey/white matter), breast tissue, and adipose tissue). The columns are ordered in decreasing density. We use the definitions of tissue substitutes provided in the [ICRU Report 44](https://www.icru.org/report/tissue-substitutes-in-radiation-dosimetry-and-measurement-report-44/) by the [International Commission on Radiation Units and Measurements](https://www.icru.org/). The material composition is available at [https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html](https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html).

### Sample composition

![](sample.png)

**Results:** The correlation coefficient between the attenuation coefficients computed with gVirtualXRay and those provided in NIST's *XCOM: Photon Cross Sections Database* is 100% for bone (cortical), blood (whole), brain (grey/white matter), breast tissue, and adipose tissue. It demonstrates that the photon cross section calculations performed in gVirtualXRay are accurate. 


The [mean absolute percentage error (MAPE)](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error), also known as mean absolute percentage deviation (MAPD), between the two simulated images is ???%. The [zero-mean normalised cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Zero-normalized_cross-correlation_(ZNCC)) is ???%. The [Structural Similarity Index (SSIM)](https://en.wikipedia.org/wiki/Structural_similarity) is ???%. As MAPE is low, and both ZNCC and SSIM are close to 100%, we can conclude that X-ray images simulated with gVirtualXRay on GPU in milliseconds are comparable to Monte Carlo simulations ran on supercomputers for days.

## Import packages

In [1]:
%matplotlib inline

import os # Locate files

import numpy as np # Who does not use Numpy?
import pandas as pd # Load/Write CSV files

import matplotlib
old_backend =  matplotlib.get_backend() 
matplotlib.use("Agg")  # Prevent showing stuff

from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table
import matplotlib.colors as mcolors

from scipy.stats import pearsonr # Compute the correlatio coefficient

from skimage.util import compare_images # Checkboard comparison between two images
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_absolute_percentage_error as mape

import SimpleITK as sitk # Load/Write MHA/MHD files

import viewscad # Use OpenSCAD to create STL files

import pyvista as pv # 3D visualisation
from pyvista import themes

import gvxrPython3 as gvxr # Simulate X-ray images

import json2gvxr # Set gVirtualXRay and the simulation up

SimpleGVXR 1.0.1 (2021-04-01T17:18:35) [Compiler: GNU g++] on Linux
gVirtualXRay core library (gvxr) 1.1.5 (2021-04-01T17:18:28) [Compiler: GNU g++] on Linux


## Reference image

We first load the reference image that has been simulated using [Gate](http://www.opengatecollaboration.org/) wrapper for CERN's [Geant4](https://geant4.web.cern.ch/).

In [2]:
# gate_image = sitk.GetArrayFromImage(sitk.ReadImage("../Gate/output/energy.mhd"))[0];
gate_image = sitk.GetArrayFromImage(sitk.ReadImage("../Gate/output/energy-total_1e10.tif"));

It is then normalised using the zero-mean, unit variance normalisation method.

In [3]:
gate_image = (gate_image - np.mean(gate_image)) / np.std(gate_image);

We plot the image using a linear look-up table and a power-law normalisation.

In [4]:
plt.figure(figsize= (20,10))

plt.suptitle("Image simulated using Gate wrapper for CERN's Geant4")

plt.subplot(121)
plt.imshow(gate_image, cmap="gray", vmin=-3, vmax=1);
plt.colorbar(orientation='horizontal');
plt.title("Using a linear colour scale");

plt.subplot(122)
plt.imshow(np.clip((gate_image + 3) / 4, 0, 1), norm=PowerNorm(gamma=1./2.), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("Using a Power-law colour scale");

plt.tight_layout()

plt.savefig('../data/reference_from_Gate.pdf')
plt.savefig('../data/reference_from_Gate.png')

![](reference_from_Gate.png)

## Setting up gVirtualXRay

Before simulating an X-ray image using gVirtualXRay, we must create an OpenGL context.

In [5]:
json2gvxr.initGVXR("simulation.json", "OPENGL");

Create an OpenGL context: 800x450


## X-ray source

We create an X-ray source. It is a point source.

In [6]:
json2gvxr.initSourceGeometry();

Set up the beam
	Source position: [0.0, 0.0, 1000.0, 'mm']
	Source shape: PointSource


## Spectrum

The beam spectrum is generated using [SpekPy](https://bitbucket.org/spekpy/spekpy_release/wiki/Home). We decided to use a beam that is clinically appropriate. The chest phantom we used corresponds to a 70kg male patient. For a male chest x-ray, the tube voltage is in the range 80-120kV depending on technique, pathology and equipment used. The total aluminium equivalent filtration is approximately 3.2mm. `verbose=1` below is used to output the beam spectrum for Gate. To disable it, change it to `verbose=0`.

In [7]:
spectrum, unit, k, f = json2gvxr.initSpectrum(verbose=1);

/gate/source/mybeam/gps/emin 11.0 keV
/gate/source/mybeam/gps/emax 90.0 keV
/gate/source/mybeam/gps/histpoint 0.011 3.0
/gate/source/mybeam/gps/histpoint 0.0115 12.0
/gate/source/mybeam/gps/histpoint 0.012 40.0
/gate/source/mybeam/gps/histpoint 0.0125 148.0
/gate/source/mybeam/gps/histpoint 0.013 519.0
/gate/source/mybeam/gps/histpoint 0.0135 1565.0
/gate/source/mybeam/gps/histpoint 0.014 4090.0
/gate/source/mybeam/gps/histpoint 0.0145 9474.0
/gate/source/mybeam/gps/histpoint 0.015 19789.0
/gate/source/mybeam/gps/histpoint 0.0155 37826.0
/gate/source/mybeam/gps/histpoint 0.016 67047.0
/gate/source/mybeam/gps/histpoint 0.0165 111380.0
/gate/source/mybeam/gps/histpoint 0.017 174907.0
/gate/source/mybeam/gps/histpoint 0.0175 261254.0
/gate/source/mybeam/gps/histpoint 0.018 373324.0
/gate/source/mybeam/gps/histpoint 0.0185 513706.0
/gate/source/mybeam/gps/histpoint 0.019 684248.0
/gate/source/mybeam/gps/histpoint 0.0195 885574.0
/gate/source/mybeam/gps/histpoint 0.02 1116396.0
/gate/source

In [8]:
print("kvp:", json2gvxr.params["Source"]["Beam"]["kvp"]);
print("filter material:", json2gvxr.params["Source"]["Beam"]["filter"][0]);
print("filter thickness (in mm):", json2gvxr.params["Source"]["Beam"]["filter"][1]); 

kvp: 90
filter material: Al
filter thickness (in mm): 3.2


Plot the spectrum

In [9]:
energy_set = sorted(spectrum.keys())
count_set = [];

for energy in energy_set:
    count_set.append(spectrum[energy])
    
plt.figure(figsize= (10,10))

plt.plot(k, f) # Plot the spectrum
plt.xlabel('Energy [keV]')
plt.ylabel('Fluence per mAs per unit energy [photons/cm2/mAs/keV]')
plt.title('X-ray spectrum')

plt.tight_layout()

plt.savefig('../data/spectrum.pdf')
plt.savefig('../data/spectrum.png')

![](spectrum.png)

## Detector

Create a digital detector

In [10]:
json2gvxr.initDetector();

Set up the detector
	Detector position: [0.0, 0.0, -125.0, 'mm']
	Detector up vector: [0, 1, 0]
	Detector number of pixels: [512, 512]
	Pixel spacing: [0.29296875, 0.29296875, 'mm']


## Sample

We now create CAD models using [OpenSCAD](https://openscad.org/) and extract the corresponding STL files.

In [11]:
openscad_make_spheres_str = """

module make_column_of(sphere_radius, height, count)
{
    step = height / (count - 1);
    for (a = [0 : count - 1]) {
        offset = -height / 2 + step * a ;
        translate([0, offset, 0])
            sphere(sphere_radius[a], $fn=50);
    }
}

module make_row_of(radius, count, id)
{
    step = radius / (count - 1);
    for (a = [0 : count - 1]) {
        if (id == -1 || id == a) {
            offset = -radius / 2 + step * a ;
            translate([offset, 0, 0])
                children();
        }
    }
}

module make_spheres(sphere_radius, ring_radius, ring_count, column_height, column_count, id = -1)
{
    make_row_of(radius = ring_radius, count = ring_count, id = id)
        make_column_of(sphere_radius, height = column_height, count = column_count);
}
"""

The matrix

In [12]:
openscad_matrix_str = """

color("red")
    difference() {
        scale([70, 70, 15])
            cube(1, center = true);
        make_spheres([2, 3.5, 5], 50, 5, 40, 3, -1);
}

"""

In [13]:
# r = viewscad.Renderer()
# r.render(openscad_matrix_str + openscad_make_spheres_str, outfile='matrix.stl')

In [14]:
openscad_cube_str = """

color("red")
    scale([70, 70, 15])
        cube(1, center = true);

"""

In [15]:
# r = viewscad.Renderer()
# r.render(openscad_cube_str, outfile='cube.stl')

The spheres

In [16]:
openscad_col_str_set = [];

for i in range(5):
    openscad_col_str_set.append("""
    color("blue")
        make_spheres([2, 3.5, 5], 50, 5, 40, 3, """ + str(i) + ");");
        
#     r = viewscad.Renderer()
#     r.render(openscad_col_str_set[-1] + openscad_make_spheres_str, outfile='col_' + str(i) + '.stl')

Load the samples. `verbose=2` is used to print the material database for Gate. To disable it, use `verbose=0` or `verbose=1`.

In [17]:
json2gvxr.initSamples(verbose=2);

Load the 3D data

Bone_Cortical_ICRU_44: d=1.92 g/cm3 ; n=9 ; state=solid
        +el: name=Hydrogen ; f=0.034
        +el: name=Carbon ; f=0.155
        +el: name=Nitrogen ; f=0.042
        +el: name=Oxygen ; f=0.435
        +el: name=Sodium ; f=0.001
        +el: name=Magnesium ; f=0.002
        +el: name=Phosphor ; f=0.103
        +el: name=Sulfur ; f=0.003
        +el: name=Calcium ; f=0.225

Blood_Whole_ICRU_44: d=1.06 g/cm3 ; n=10 ; state=solid
        +el: name=Hydrogen ; f=0.102
        +el: name=Carbon ; f=0.11
        +el: name=Nitrogen ; f=0.033
        +el: name=Oxygen ; f=0.745
        +el: name=Sodium ; f=0.001
        +el: name=Phosphor ; f=0.001
        +el: name=Sulfur ; f=0.002
        +el: name=Chlorine ; f=0.003
        +el: name=Potassium ; f=0.002
        +el: name=Iron ; f=0.001

Brain_Grey_White_Matter_ICRU_44: d=1.04 g/cm3 ; n=9 ; state=solid
        +el: name=Hydrogen ; f=0.107
        +el: name=Carbon ; f=0.145
        +el: name=Nitrogen ; f=0.022
        +el

## Mass attenuation coefficients

Before computing an X-ray image, we can check that the mass attenuation coefficients are accurate. We downloaded tabulated data from the XCOM database and compare the values with the ones used in gVirtualXRay. We do it for relevant chemical elements from [https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html](https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html):

- [Carbon](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z06.html)
- [Chlorine](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z17.html)
- [Hydrogen](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z01.html)
- [Iron](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z26.html)
- [Magnesium](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z12.html)
- [Nitrogen](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z07.html)
- [Oxygen](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z08.html)
- [Phosphorus](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z15.html)
- [Potassium](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z19.html)
- [Sodium](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z11.html)
- [Sulfur](https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z16.html)

In [18]:
elements = ["Carbon", "Chlorine", "Hydrogen", "Iron", "Magnesium", "Nitrogen", "Oxygen", "Potassium", "Sodium", "Sulfur"] #  "Phosphorus"]#, ];

i = 1

plt.figure(figsize= (20,40))

for element in elements:
    if os.path.isfile("../data/" + element + ".csv"):
        df = pd.read_csv("../data/" + element + ".csv")
        
        gvxr_mu_rho = []
        gvxr_energy = []
        for energy_id in range(len(df["Photon in MeV"])):
            if energy_id == len(df["Photon in MeV"]) - 1 or energy_id == 0:
                energy = df["Photon in MeV"][energy_id]
            elif df["Photon in MeV"][energy_id] == df["Photon in MeV"][energy_id + 1]:
                energy = df["Photon in MeV"][energy_id] - df["Photon in MeV"][energy_id] * 1e-3
            elif df["Photon in MeV"][energy_id] == df["Photon in MeV"][energy_id - 1]:
                energy = df["Photon in MeV"][energy_id] + df["Photon in MeV"][energy_id] * 1e-3
            else:
                energy = df["Photon in MeV"][energy_id]
            
            gvxr_mu_rho.append(gvxr.getMassAttenuationFromElement(element, energy, "MeV"))
            gvxr_energy.append(energy)
        
        df["Mass attenuation coefficient (gVirtualXRay)"] = gvxr_mu_rho
        df["|XCOM - gVirtualXRay| / XCOM (in %)"] = np.abs(np.array(100 * (df["Mass attenuation coefficient"] - gvxr_mu_rho) / df["Mass attenuation coefficient"])).astype(int)
        
        if df["|XCOM - gVirtualXRay| / XCOM (in %)"].mean() > 1.0:
            print("\tWARNING:")
            print("\t\tAverage:", df["|XCOM - gVirtualXRay| / XCOM (in %)"].mean())
            print("\t\tMax:", df["|XCOM - gVirtualXRay| / XCOM (in %)"].abs().max())
        
        df.to_csv("../data/" + element + "-validation.csv", index=False )
        
        plt.subplot(6, 2, i)
        plt.title("Mass attenuation in mu / rho:  " + element)
        plt.plot(df["Photon in MeV"], df["Mass attenuation coefficient"], label="XCOM", color="b")
        plt.scatter(gvxr_energy, gvxr_mu_rho, label="gVirtualXRay", s=15, color="r")
        plt.legend()
        plt.xscale('log')
        plt.yscale('log')
        
        i = i + 1
        
plt.tight_layout()

plt.savefig('../data/mass_attenuation_coefficients_from_elements.pdf')
plt.savefig('../data/mass_attenuation_coefficients_from_elements.png')

![](mass_attenuation_coefficients_from_elements.png)

And for some clinically relevant materials from [https://physics.nist.gov/PhysRefData/XrayMassCoef/tab4.html](https://physics.nist.gov/PhysRefData/XrayMassCoef/tab4.html):

- [Adipose Tissue (ICRU-44)](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/adipose.html)
- [Blood, Whole (ICRU-44)](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/blood.html)
- [Bone, Cortical (ICRU-44)](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/bone.html)
- [Brain, Grey/White Matter (ICRU-44)](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/brain.html)
- [Breast Tissue (ICRU-44)](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/breast.html)
- [Water, Liquid](https://physics.nist.gov/PhysRefData/XrayMassCoef/ComTab/water.html)

In [19]:
i = 1

plt.figure(figsize= (20,20))

for sample in json2gvxr.params["Samples"]:
    label = sample["Label"]
    if os.path.isfile("../data/" + label + ".csv"):
        df = pd.read_csv("../data/" + label + ".csv")
        
        gvxr_mu_rho = []
        gvxr_energy = []

        for energy_id in range(len(df["Photon in MeV"])):
            if energy_id == len(df["Photon in MeV"]) - 1 or energy_id == 0:
                energy = df["Photon in MeV"][energy_id]
            elif df["Photon in MeV"][energy_id] == df["Photon in MeV"][energy_id + 1]:
                energy = df["Photon in MeV"][energy_id] - df["Photon in MeV"][energy_id] * 1e-3
            elif df["Photon in MeV"][energy_id] == df["Photon in MeV"][energy_id - 1]:
                energy = df["Photon in MeV"][energy_id] + df["Photon in MeV"][energy_id] * 1e-3
            else:
                energy = df["Photon in MeV"][energy_id]

            gvxr_mu_rho.append(gvxr.getMassAttenuationCoefficient(label, energy, "MeV"))
            gvxr_energy.append(energy)

        
        df["Mass attenuation coefficient (gVirtualXRay)"] = gvxr_mu_rho
        df["|XCOM - gVirtualXRay| / XCOM (in %)"] = np.abs(np.array(100 * (df["Mass attenuation coefficient"] - gvxr_mu_rho) / df["Mass attenuation coefficient"])).astype(int)
        
        if df["|XCOM - gVirtualXRay| / XCOM (in %)"].mean() > 1.0:
            print("\tWARNING:")
            print("\t\tAverage:", df["|XCOM - gVirtualXRay| / XCOM (in %)"].mean())
            print("\t\tMax:", df["|XCOM - gVirtualXRay| / XCOM (in %)"].abs().max())

        df.to_csv("../data/" + label + "-validation.csv", index=False)
        
        plt.subplot(3, 2, i)
        plt.title("Mass attenuation in mu / rho:  " + label)
        plt.plot(df["Photon in MeV"], df["Mass attenuation coefficient"], label="XCOM", color="b")
        plt.scatter(gvxr_energy, gvxr_mu_rho, label="gVirtualXRay", s=15, color="r")
        plt.legend()
        plt.xscale('log')
        plt.yscale('log')
        
        corr, _ = pearsonr(df["Mass attenuation coefficient"], gvxr_mu_rho)
        print(label + '\'s Pearsons correlation: %.3f' % corr)
        
        i = i + 1
        
plt.tight_layout()

plt.savefig('../data/mass_attenuation_coefficients_from_ICRU44.pdf')
plt.savefig('../data/mass_attenuation_coefficients_from_ICRU44.png')

H2O's Pearsons correlation: 1.000
Bone_Cortical_ICRU_44's Pearsons correlation: 1.000
Blood_Whole_ICRU_44's Pearsons correlation: 1.000
Brain_Grey_White_Matter_ICRU_44's Pearsons correlation: 1.000
Breast_Tissue_ICRU_44's Pearsons correlation: 1.000
Adipose_Tissue_ICRU_44's Pearsons correlation: 1.000


![](mass_attenuation_coefficients_from_ICRU44.png)

## Runing the simulation

Update the 3D visualisation and take a screenshot

In [20]:
gvxr.displayScene();

gvxr.useLighing();
gvxr.useWireframe();
gvxr.setZoom(719.6787109375);
gvxr.setSceneRotationMatrix([0.7624880075454712, 0.09040657430887222, -0.6406543850898743, 0.0,
                             0.05501500517129898, 0.9775413870811462, 0.20342488586902618, 0.0,
                             0.6446591019630432, -0.190354123711586, 0.7403913140296936, 0.0,
                             0.0, 0.0, 0.0, 1.0]);

gvxr.displayScene();

In [21]:
screenshot = gvxr.takeScreenshot();

In [22]:
plt.figure(figsize= (10,10))
plt.title("Screenshot");
plt.imshow(screenshot);
plt.axis('off');

plt.tight_layout()

plt.savefig('../data/screenshot-beam-off.pdf')
plt.savefig('../data/screenshot-beam-off.png')

![](screenshot-beam-off.png)

Compute an X-ray image and add it to the list of projections

In [23]:
x_ray_image = gvxr.computeXRayImage();

Save the corresponding image

In [24]:
volume = sitk.GetImageFromArray(x_ray_image);
sitk.WriteImage(volume, 'projection.mha');

Normalise the data

In [25]:
x_ray_image = (x_ray_image - np.mean(x_ray_image)) / np.std(x_ray_image);

## Compute the L-buffers then the energy fluence

Compute the L-buffers

In [26]:
NoneType = type(None)
import copy

L_buffer_set = {};

L_buffer_inner = None;
L_buffer_outer = None;

for sample in json2gvxr.params["Samples"]:
    L_buffer_set[sample["Label"]] = np.array(gvxr.computeLBuffer(sample["Label"]));
    
    if sample["Type"] == "inner":
        
        volume = sitk.GetImageFromArray(L_buffer_set[sample["Label"]]);
        sitk.WriteImage(volume, 'L-buffer_' + sample["Label"] + '.mha');
    
        if isinstance(L_buffer_inner, NoneType):
            L_buffer_inner = copy.deepcopy(L_buffer_set[sample["Label"]])
        else:
            L_buffer_inner += L_buffer_set[sample["Label"]]
    else:
        L_buffer_outer = sample["Label"]

if not isinstance(L_buffer_outer, NoneType):
    L_buffer_set[L_buffer_outer] -= L_buffer_inner
    
    volume = sitk.GetImageFromArray(L_buffer_set[L_buffer_outer]);
    sitk.WriteImage(volume, 'L-buffer_' + L_buffer_outer + '.mha');

Integrate the energy for every energy channe;

In [27]:
x_ray_image_manual = np.zeros(x_ray_image.shape);

for energy, count in zip(energy_set, count_set):

    mu_x = np.zeros(x_ray_image.shape);

    for sample in json2gvxr.params["Samples"]:
        mu = gvxr.getLinearAttenuationCoefficient(sample["Label"], energy, "keV");
        mu_x += L_buffer_set[sample["Label"]] * mu;

    x_ray_image_manual += (energy / 1000.0 * count) * np.exp(-mu_x)

Save the corresponding image

In [28]:
volume = sitk.GetImageFromArray(x_ray_image_manual);
sitk.WriteImage(volume, 'projection_manual.mha');

Normalise the data

In [29]:
x_ray_image_response = (x_ray_image_manual - np.mean(x_ray_image_manual)) / np.std(x_ray_image_manual);

## Model the camera response

Load the response

In [30]:
detector_response = np.loadtxt("../Gate/data/responseDetector.txt");

Display the response

In [31]:
plt.figure(figsize= (10,10))
plt.title("Detector response");
plt.plot(detector_response[:,0] * 1000, detector_response[:,1] * 1000);
plt.xlabel('Incident energy: E (in keV)');
plt.ylabel('detector response: δ(E) (in keV)');

plt.tight_layout()

plt.savefig('../data/detector_response.pdf')
plt.savefig('../data/detector_response.png')

![](detector_response.png)

Integrate the energy for every energy channe;

In [32]:
def interpolate(a_low, a_high, a0, b_low, b_high):
    return b_low + (b_high - b_low) * (a0 - a_low) / (a_high - a_low)

def find_nearest(a, a0, b):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    
    # a[idx] <= a0 <= a[idx+1]
    if a[idx] < a0:
        return interpolate(a[idx], a[idx + 1], a0, b[idx], b[idx + 1])
    # a[idx - 1] <= a0 <= a[idx]
    else:
        return interpolate(a[idx - 1], a[idx], a0, b[idx - 1], b[idx])

In [33]:
x_ray_image_response = np.zeros(x_ray_image.shape);

for energy, count in zip(energy_set, count_set):

    mu_x = np.zeros(x_ray_image.shape);

    for sample in json2gvxr.params["Samples"]:
        mu = gvxr.getLinearAttenuationCoefficient(sample["Label"], energy, "keV");
        mu_x += L_buffer_set[sample["Label"]] * mu;

    corrected_energy = find_nearest(detector_response[:,0], energy / 1000, detector_response[:,1])

    x_ray_image_response += (corrected_energy * count) * np.exp(-mu_x)

Save the corresponding image

In [34]:
volume = sitk.GetImageFromArray(x_ray_image_response);
sitk.WriteImage(volume, 'x_ray_image_response.mha');

Normalise the data

In [35]:
x_ray_image_response = (x_ray_image_response - np.mean(x_ray_image_response)) / np.std(x_ray_image_response);

## Interactive 3D visualisation

In [36]:
# %matplotlib widget

# my_theme = themes.DarkTheme()
# my_theme.color = 'red'
# my_theme.lighting = False
# my_theme.show_edges = True
# my_theme.axes.box = True

# pv.global_theme.load_theme(my_theme)

plotter = pv.Plotter(notebook=True, window_size=(512 * 4, 256 * 4))
plotter.add_title("3D Visualisation")
i = 0;

colours = list(mcolors.TABLEAU_COLORS);
colour_id = 0;

# Add the samples
for sample in json2gvxr.params["Samples"]:
   
    if sample["Label"] == "H2O":
        opacity = 0.25;
    else:
        opacity = 1.0;

    mesh = pv.read(sample["Path"])

    colour = mcolors.to_rgb(colours[colour_id]);
    plotter.add_mesh(mesh, opacity=opacity, color=(colour[0], colour[1], colour[2]))
    colour_id += 1

# Add the source
source_position = json2gvxr.params["Source"]["Position"];
source_position = np.array([source_position[0], source_position[1], source_position[2]])

sphere = pv.Sphere(radius=10, center=source_position)
colour = mcolors.to_rgb(colours[colour_id]);
plotter.add_mesh(sphere, color=(colour[0], colour[1], colour[2]))
colour_id += 1

# Add the detector
detector_position = json2gvxr.params["Detector"]["Position"];
detector_position = np.array([detector_position[0], detector_position[1], detector_position[2]])

detector_direction = detector_position - source_position;
plane = pv.Plane(center=detector_position,
                 direction=detector_direction,
                 i_size=json2gvxr.params["Detector"]["Size"][0],
                 j_size=json2gvxr.params["Detector"]["Size"][1],
                 i_resolution=1, j_resolution=1)

# Creating a custom RGBA image
cmap = get_cmap("gray")
norm = lambda x: (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))
hue = norm(np.fliplr(x_ray_image_response.T).ravel())
colors = (cmap(hue) * 255.0).astype(np.uint8)
image = colors.reshape((x_ray_image_response.shape[0], x_ray_image_response.shape[1], 4), order="F")

texture = pv.numpy_to_texture(image)
plotter.add_mesh(plane, texture=texture)

cpos = plotter.show(cpos=[(-965.0666772245627, 47.72249675199398, 1249.8218575579394),
                          (0.0, 0.0, 442.5),
                          (0, 1, 0)], screenshot=True)

ViewInteractiveWidget(height=1024, layout=Layout(height='auto', width='100%'), width=2048)

Update the 3D visualisation and take a screenshot

In [37]:
gvxr.displayScene();
screenshot = gvxr.takeScreenshot();

In [38]:
plt.figure(figsize= (10,10))
plt.title("Screenshot");
plt.imshow(screenshot);
plt.axis('off');

plt.tight_layout()

plt.savefig('../data/screenshot-beam-on.pdf')
plt.savefig('../data/screenshot-beam-on.png')

![](screenshot-beam-on.png)

Plot the X-ray image

In [39]:
plt.figure(figsize= (20,10))
plt.suptitle("Image simulated using gVirtualXRay")

plt.subplot(121)
plt.imshow(x_ray_image, cmap="gray", vmin=-3, vmax=1);
plt.colorbar(orientation='horizontal');
plt.title("Using a linear colour scale");

plt.subplot(122)
plt.imshow(np.clip((x_ray_image + 3) / 4, 0, 1), norm=PowerNorm(gamma=1./2.), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("Using a Power-law colour scale");

plt.tight_layout()

plt.savefig('../data/simulated_xray_gVirtualXRay.pdf')
plt.savefig('../data/simulated_xray_gVirtualXRay.png')

![](simulated_xray_gVirtualXRay.png)

## Comparison the analytic simulation with the Monte Carloa simulation

### Quantitative validation

Compute image metrics between the two simulated images:

1. [mean absolute percentage error (MAPE)](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error), also known as mean absolute percentage deviation (MAPD),
2. [zero-mean normalised cross-correlation (ZNCC)](https://en.wikipedia.org/wiki/Cross-correlation#Zero-normalized_cross-correlation_(ZNCC)), and 
3. [Structural Similarity Index (SSIM)](https://en.wikipedia.org/wiki/Structural_similarity).

In [40]:
MAPE = mape(gate_image, x_ray_image);
ZNCC = np.mean(gate_image * x_ray_image);
SSIM = ssim(gate_image, x_ray_image, data_range=gate_image.max() - gate_image.min());

print("MAPE:", "{0:0.2f}".format(100 * MAPE) + "%")
print("ZNCC:", "{0:0.2f}".format(100 * ZNCC) + "%")
print("SSIM:", "{0:0.2f}".format(100 * SSIM) + "%")

MAPE: 7.29%
ZNCC: 99.36%
SSIM: 91.59%


### Qualitative validation

Checkboard comparison

In [41]:
absolute_error = np.abs(gate_image - x_ray_image_response);
comp_equalized = compare_images(gate_image, x_ray_image_response, method='checkerboard', n_tiles=(16,16))

plt.figure(figsize= (20,10))

plt.subplot(141)
plt.imshow(x_ray_image_response, cmap="gray", vmin=-3, vmax=1);
plt.title("gVirtualXRay");

plt.subplot(142)
plt.imshow(gate_image, cmap="gray", vmin=-3, vmax=1);
plt.title("Gate");

plt.subplot(143)
plt.imshow(comp_equalized, cmap="gray", vmin=-3, vmax=1);
plt.title("gVirtualXRay & Gate");

plt.subplot(144)
plt.imshow(absolute_error, cmap="gray", vmin=0, vmax=1);
plt.title("Absolute error");

plt.tight_layout()

plt.savefig('../data/comparison.pdf')
plt.savefig('../data/comparison.png')

![](comparison.png)

Plot the profiles

In [42]:
plt.figure(figsize= (10,20))

offset_line = 20 * (json2gvxr.params["Source"]["Position"][2] - json2gvxr.params["Detector"]["Position"][2]) / json2gvxr.params["Source"]["Position"][2]

spacing = json2gvxr.params["Detector"]["Size"][0] / json2gvxr.params["Detector"]["NumberOfPixels"][0];

x = np.arange(0.0, json2gvxr.params["Detector"]["Size"][0], spacing)
print(x.shape)

plt.subplot(311)
plt.title("Profiles (Top line)");
# plt.yscale("log")
y_coord = round(json2gvxr.params["Detector"]["NumberOfPixels"][0] / 2 - offset_line * json2gvxr.params["Detector"]["NumberOfPixels"][0] / json2gvxr.params["Detector"]["Size"][0])
plt.plot(x, x_ray_image_response[y_coord], label="gVirtualXRay")
plt.plot(x, gate_image[y_coord], label="Gate")
plt.xlabel("Pixel location (in mm)")
plt.ylabel("Pixel intensity")
plt.legend()

plt.subplot(312)
plt.title("Profiles (Central line)");
# plt.yscale("log")
y_coord = round(json2gvxr.params["Detector"]["NumberOfPixels"][0] / 2)
plt.plot(x, x_ray_image_response[y_coord], label="gVirtualXRay")
plt.plot(x, gate_image[y_coord], label="Gate")
plt.xlabel("Pixel location (in mm)")
plt.ylabel("Pixel intensity")

plt.subplot(313)
plt.title("Profiles (Bottom line)");
# plt.yscale("log")
y_coord = round(json2gvxr.params["Detector"]["NumberOfPixels"][0] / 2 + offset_line * json2gvxr.params["Detector"]["NumberOfPixels"][0] / json2gvxr.params["Detector"]["Size"][0])
plt.plot(x, x_ray_image_response[y_coord], label="gVirtualXRay")
plt.plot(x, gate_image[y_coord], label="Gate")
plt.xlabel("Pixel location (in mm)")
plt.ylabel("Pixel intensity")

plt.tight_layout()

plt.savefig('../data/profiles.pdf')
plt.savefig('../data/profiles.png')

(512,)


![](profiles.png)

Destroy the window

In [43]:
# gvxr.destroyAllWindows()